In [77]:

import time

In [ ]:
import requests
import json
# import pandas as pd
# from datetime import datetime

# API Configuration
BASE_URL = "http://localhost:8001"
# BASE_URL = "http://hackathon-demo-80967351.us-west-2.elb.amazonaws.com"
HEADERS = {"Content-Type": "application/json"}

def post_request(endpoint: str, payload: dict, headers: dict, timeout: int = 15):
    url = f"{BASE_URL}/{endpoint}"
    print(url)
    
    try:
        response = requests.post(
            url,
            json=payload,
            headers=headers,
            timeout=timeout
        )
        
        print(f"\n📡 POST to {endpoint}")
        print(f"Status Code: {response.status_code}")
        print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Request successful!")
            return result
        else:
            print(f"❌ Request failed: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None

## Product matching (With Image)

In [109]:
def product_matching(caption_payload, user_messages):
    start = time.time()

    ## Image Captioning request
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("find docs...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference

In [ ]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

product_matching(caption_payload, user_messages)

image captioning...
http://44.247.1.37:8001/image_captioning

📡 POST to image_captioning
Status Code: 200
Response Time: 2.07s
✅ Request successful!
find docs...
http://44.247.1.37:8001/finding_documents

📡 POST to finding_documents
Status Code: 200
Response Time: 1.81s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean reco

("For a beach party, I recommend exploring some lovely floral maxi dresses or breezy off-shoulder sundresses. They'll keep you looking cute and feeling comfortable. For your formal dinner, our elegant belted midi dresses or sophisticated satin slip dresses would be perfect. I suggest browsing through our search results to find the exact style that speaks to you. Each dress has its own unique charm, so you're sure to find something that makes you feel amazing!",
 [{'search_term': 'floral maxi dress',
   'search_results': [{'score': 0.7890574,
     'id': 'Image40.jpg',
     'name': 'Yellow Floral Spaghetti Strap Midi Dress',
     'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe:

In [101]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }
product_matching(caption_payload, user_messages)

No image provided for captioning.
find docs...
http://44.247.1.37:8001/finding_documents

📡 POST to finding_documents
Status Code: 200
Response Time: 1.55s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean recommended or delicate hand wash. Matches Well With: Outerwear: A light white cardigan or a cropped denim jacket. Shoe

("For a beach party, I recommend exploring some lovely floral maxi dresses or off-shoulder sundresses that will keep you looking cute and feeling comfortable. They're perfect for a relaxed summer event. For your formal dinner, our elegant belted midi dresses or sophisticated satin-style options would be stunning. I suggest browsing through our search results to find the exact piece that catches your eye. Would you like me to help you narrow down some specific styles?",
 [{'search_term': 'floral maxi dress',
   'search_results': [{'score': 0.7890574,
     'id': 'Image40.jpg',
     'name': 'Yellow Floral Spaghetti Strap Midi Dress',
     'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Sty

### Product Matching (Without Image)

In [102]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

## Style matching

In [103]:
def style_matching(caption_payload, user_messages):
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("style complement...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }
    result = post_request("style_complement", doc_payload, HEADERS, timeout=30)
    style_complement = result['results']
    print(style_complement)
    print("find docs...")
    doc_payload = {
        "user_query": style_complement,
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference
        

In [104]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

In [105]:
style_matching(caption_payload, user_messages)

image captioning...
http://44.247.1.37:8001/image_captioning

📡 POST to image_captioning
Status Code: 200
Response Time: 1.63s
✅ Request successful!
style complement...
http://44.247.1.37:8001/style_complement

📡 POST to style_complement
Status Code: 200
Response Time: 2.76s
✅ Request successful!
[
    "summer linen blazer",
    "strappy heeled sandals neutral",
    "minimalist statement earrings",
    "lightweight silk scarf",
    "structured leather tote bag",
    "tailored wide-leg trousers",
    "sleek ankle boots"
]
find docs...
http://44.247.1.37:8001/finding_documents

📡 POST to finding_documents
Status Code: 200
Response Time: 2.69s
✅ Request successful!
[{'search_term': 'summer linen blazer', 'search_results': [{'score': 0.7474082, 'id': 'Image22.png', 'name': 'Sunny Yellow Linen Shirt', 'description': 'This vibrant sunny yellow linen shirt is designed for a relaxed and airy fit, perfect for warm weather. It features a classic collar and a single chest pocket, with the ability

("I apologize, but it seems like the last message was cut off. Could you complete your request or clarify what specific items you're looking for? Based on the summer product information I see, I can suggest some versatile pieces like a lightweight linen shirt in a sunny yellow, comfortable wide-leg trousers in a neutral beige, or some chic flat sandals that would work well for different summer occasions. Would you like me to elaborate on any of those?",
 [{'search_term': 'summer linen blazer',
   'search_results': [{'score': 0.7474082,
     'id': 'Image22.png',
     'name': 'Sunny Yellow Linen Shirt',
     'description': 'This vibrant sunny yellow linen shirt is designed for a relaxed and airy fit, perfect for warm weather. It features a classic collar and a single chest pocket, with the ability to be styled loosely or tucked in. The breathable linen fabric ensures comfort and a casually sophisticated look. Style Vibe: Resort Casual / Effortless Summer. Fit: Relaxed, slightly oversized

In [106]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

style_matching(caption_payload, user_messages)

No image provided for captioning.
style complement...
http://44.247.1.37:8001/style_complement

📡 POST to style_complement
Status Code: 200
Response Time: 2.84s
✅ Request successful!
Since no image was provided, I'll generate a generic summer fashion search query list based on the conversation context:

[
    "summer beach party dresses",
    "casual chic sundresses",
    "off-shoulder beach outfit",
    "satin slip dress formal dinner",
    "belted midi dress evening wear",
    "summer cocktail attire",
    "resort casual clothing"
]
find docs...
http://44.247.1.37:8001/finding_documents

📡 POST to finding_documents
Status Code: 200
Response Time: 1.99s
✅ Request successful!
[{'search_term': 'summer beach party dresses', 'search_results': [{'score': 0.6758316, 'id': 'Image22.png', 'name': 'Sunny Yellow Linen Shirt', 'description': 'This vibrant sunny yellow linen shirt is designed for a relaxed and airy fit, perfect for warm weather. It features a classic collar and a single chest poc

("For a beach party, I recommend exploring some fun and breezy options like a tropical print sarong skirt paired with a lightweight top, or a bright swing mini dress that's perfect for a casual, playful look. For your formal dinner, a dusty rose pleated midi dress with a cinched waist would be elegant and sophisticated. I suggest browsing through the search results to find the exact pieces that catch your eye. Would you like me to help you narrow down your choices?",
 [{'search_term': 'summer beach party dresses',
   'search_results': [{'score': 0.6758316,
     'id': 'Image22.png',
     'name': 'Sunny Yellow Linen Shirt',
     'description': 'This vibrant sunny yellow linen shirt is designed for a relaxed and airy fit, perfect for warm weather. It features a classic collar and a single chest pocket, with the ability to be styled loosely or tucked in. The breathable linen fabric ensures comfort and a casually sophisticated look. Style Vibe: Resort Casual / Effortless Summer. Fit: Relaxe

### testing

In [112]:
## Setting input
caption_payload = {
    "image_path": "Data/thaitrouser.png"
}


user_messages = {
        "messages": [
            {"role": "user", "content": "I want a shirt that goes well with my Thai trousers."},
        ]
    }

style_matching(caption_payload, user_messages)

image captioning...
http://hackathon-demo-80967351.us-west-2.elb.amazonaws.com/image_captioning

📡 POST to image_captioning
Status Code: 200
Response Time: 2.94s
✅ Request successful!
style complement...
http://hackathon-demo-80967351.us-west-2.elb.amazonaws.com/style_complement

📡 POST to style_complement
Status Code: 200
Response Time: 2.17s
✅ Request successful!
[
    "white linen button-down shirt",
    "solid color crop top thai style",
    "neutral cotton tank top",
    "minimalist silk blouse",
    "fitted white cotton shirt",
    "off-shoulder bohemian top",
    "lightweight chambray shirt"
]
find docs...
http://hackathon-demo-80967351.us-west-2.elb.amazonaws.com/finding_documents

📡 POST to finding_documents
Status Code: 200
Response Time: 1.30s
✅ Request successful!
[{'search_term': 'patterned pants', 'search_results': [{'score': 0.7374154, 'id': 'Image28.png', 'name': 'Olive Green High-Waisted Cargo Pants', 'description': 'These olive green cargo pants combine a flattering h

("Since you're looking for a top to pair with Thai trousers, I'd recommend something lightweight and simple that complements the relaxed, flowing style of those pants. A cropped top or a fitted tank could work beautifully, especially in a neutral color like black or white. You might also consider a loose, breathable blouse that creates a nice contrast with the trousers. I suggest exploring the search results to find something that matches your personal style and the specific cut of your Thai trousers.",
 [{'search_term': 'patterned pants',
   'search_results': [{'score': 0.7374154,
     'id': 'Image28.png',
     'name': 'Olive Green High-Waisted Cargo Pants',
     'description': 'These olive green cargo pants combine a flattering high-waisted fit with edgy, utilitarian details. They feature multiple large pockets, a prominent chain detail, and adjustable straps, creating a bold and statement-making look. The tapered leg finishes in a comfortable elasticated cuff, perfect for pairing wi